# Obtención de los conjuntos X e Y para el entrenamiento del modelo

In [ ]:
import os
import cv2
from glob import glob
import pandas as pd
import numpy as np

In [ ]:
def load_images_from_folder (folder):
    '''Devuelve las imágenes contenidas en una carpeta'''
    
    images = []

    for filename in sorted(os.listdir(folder)):

        if filename.endswith(".jpg"):
            # Para tener solo un canal inidicamos cv2.IMREAD_GRAYSCALE en la lectura de la imagen.
            img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)

            if img is not None:
                images.append(img)

    return images

In [ ]:
def is_label_file(filename):
    '''Indica si se trata del fichero de labels de saltos de línea.'''

    if (filename.endswith(".csv")):
            return True

    return False

In [ ]:
# Indicamos la carpeta de la que obtener los vídeos.
parent_folder = "/Users/equipo-ojo/Documents/OJO/video_inputs/*"

# Obtenemos el listado de carpetas de videos.
parent_video_folders_list = (sorted(glob(parent_folder, recursive = True)))

In [ ]:
# Obtenemos el listado de carpetas de segundos que están en una carpeta de video.
video_folders = []
breaklines_per_video = []

for folder in parent_video_folders_list:
    breaklines_per_video.extend(pd.read_csv(f"{folder}{os.path.sep}{folder.split('frames_')[1]}.csv")["seconds"].tolist())
    video_folders.append(sorted(glob(f"{folder}{os.path.sep}*", recursive = True)))

In [ ]:
# Obtenemos el lostado de imágenes correspondientes a los frames de cada segundo de vídeo, ordenado.
listado_imgs = []

for video_folder in video_folders:

    for i in range(len(video_folder)):
        frames_folder = video_folder[i]

        if is_label_file(frames_folder):
            continue

        listado_imgs.append(load_images_from_folder(frames_folder))

In [ ]:
# Cargamos el listado de imágenes en un dataframe para poder convertirlo a numpy_array.
df = pd.DataFrame(data=listado_imgs)

column_names = dict((lambda x: (x, f"Img{x}"))(i) for i in range(len(df)))
df.rename(columns=column_names, inplace=True)

listado_imgs_numpy_array = np.array(df.values.tolist())

# Comprobamos las dimensiones del numpy_array (debería ser de la forma (1249, 10, 80, 80)).
print(listado_imgs_numpy_array.shape)

In [ ]:
# Comprobamos la aparición de nulos (sería debido a carpetas que tenían menos de 10 frames).

for i in range(len(df)):
    print(f"Img{i}: {df[f'Img{i}'].isnull().values.sum()}")

In [ ]:
# Obtenemos nuestros conjuntos de entrada al modelo.

x_set = listado_imgs_numpy_array # Nuestros conjuntos de 10 frames como variables independientes.
y_set = breaklines_per_video # Nuestro label (variable dependiente)
y_set=np.array(y_set[:len(x_set)]) # Acortamos y_set para que tenga las misma longitud que x_set.
                                   # (si está bien hecho, esto no aplica)

In [ ]:
# Comprobamos las dimensiones de los conjuntos obtenidos, éstas deben coincidir en el primer nivel.

print(f"Dimensiones del conjunto total: x_set={x_set.shape},dimensiones y_set={len(y_set)}")

In [ ]:
# Guardamos los conjuntos en un fichero local para su posterior uso.
from numpy import asarray
from numpy import save

save('/Users/equipo-ojo/Documents/OJO/video_inputs/x_set_left_1.npy', x_set)
save('/Users/equipo-ojo/Documents/OJO/video_inputs/y_set_left_1.npy', y_set)